In [1]:
# pip install mediapipe

In [2]:
# pip install pyspellchecker

In [3]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
from tensorflow.keras.models import load_model
from pathlib import Path
from glob import glob
from spellchecker import SpellChecker


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [5]:
# MODEL_DIR = './aslalphabetcnnmodel1'
# MODEL_PATH = MODEL_DIR + '/ann-model98.h5'
# MODEL_WEIGHTS_PATH = MODEL_DIR + '/ann-model98.weights.h5'


MODEL_DIR = './akram/Models'
MODEL_WEIGHTS_DIR = './akram/Weights'
MODEL_PATH = MODEL_DIR + '/ann-model-99.35.h5'
MODEL_WEIGHTS_PATH = MODEL_WEIGHTS_DIR + '/ann-model-99.35.weights.h5'
CLASSES = ['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 ' ',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']
spell = SpellChecker()

In [6]:
# def preprocess_image(image):
#     sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)
#     return sobely

In [7]:
def load_model_from_disk():
    '''A convenience method for re-running certain parts of the
    analysis locally without refitting all the data.'''
    model_file = Path(MODEL_PATH)
    model_weights_file = Path(MODEL_WEIGHTS_PATH)
    
    print(model_weights_file)
    
                      
    if model_file.is_file() and model_weights_file.is_file():
        print('Retrieving model from disk...')
        model = load_model(model_file.__str__())
                      
        print('Loading ANN model weights from disk...')
        model.load_weights(model_weights_file)
        print("=== modelloaded===")
        return model
    return None



In [8]:
signDetector = load_model_from_disk()

akram\Weights\ann-model-99.35.weights.h5
Retrieving model from disk...
Loading ANN model weights from disk...
=== modelloaded===


In [73]:
"akram"

'akram'

In [87]:
# cap = cv2.VideoCapture("ASL_test_1.mp4")
cap = cv2.VideoCapture(0)
_, frame = cap.read()
h, w, c = frame.shape

i = 0
text = ''

with mp_hands.Hands(min_detection_confidence=0.4, min_tracking_confidence=0.3,max_num_hands=1) as hands: 
    while cap.isOpened():
        i = i + 1
        ret, frame = cap.read()
        last_char= ""
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        #j'ai enlever cette instruction pour esseyer d'ajuster la video que j'ai enregistrer suivant le model
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        
        croppedImage = cv2.resize(image,(600, 600))
        
        results = hands.process(croppedImage)
      
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        image = cv2.cvtColor(croppedImage,cv2.COLOR_RGB2BGR)
        
        font = cv2.FONT_HERSHEY_SIMPLEX
  
        # org
        org = (50, 50)

        # fontScale
        fontScale = 1

        # Blue color in BGR
        color = (255, 0, 0)

        # Line thickness of 2 px
        thickness = 2
        
        # Detections
        # print(results)
        
        x = []
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
#                 x_max = 0
#                 y_max = 0
#                 x_min = w
#                 y_min = h
                landmarksForImage = [[lm.x,lm.y,lm.z] for lm in hand.landmark]   
#                 x, y = int(lm.x * w), int(lm.y * h)
#                 if x > x_max:
#                     x_max = x
#                 if x < x_min:
#                     x_min = x
#                 if y > y_max:
#                     y_max = y
#                 if y < y_min:
#                     y_min = y
#                 cv2.rectangle(image, (x_min - 25, y_min - 25), (x_max + 25, y_max + 25), (0, 255, 0), 2)
#                 img1 = image[x_min:x_max,y_min:y_max]               
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                       )
            x.append(landmarksForImage)
            if i % 10 == 0 :
    #                 img = preprocess_image(img1)
    #                 croppedImage = cv2.resize(img,(64, 64))
    #                 prediction = signDetector.predict(x.reshape(-1, 64, 64, 3))
                prediction = signDetector.predict(x)   
                r = np.argmax(prediction, axis=1)
                if (CLASSES[r[0]] == " " and prediction[0][np.argmax(prediction)] >0.8):
                    print("espace")
                    txt = text.split("  ")
                    text = ""
                    for j in txt:
                        if j!="" and len(j)>1:   
                            print("passer par le correcteur ",j)
                            text = text + spell.correction(j) + "  "
                            print( spell.correction(j))
                        elif len(j)==1: 
                            text = text + j + "  "

                else :
                    if  prediction[0][np.argmax(prediction)] >0.6: 

                        if (CLASSES[r[0]]!=text[len(text)-1:]): 
                            text = text + CLASSES[r[0]]
    #                         print(CLASSES[r[0]])
                   
                    
                    print(text)
        
        cv2.putText(image,text,org, font, fontScale, color, thickness, cv2.LINE_AA)

        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()






H
H
HE
HE
HEL
HELC
espace
passer par le correcteur  HELC
help
espace
passer par le correcteur  help
help
espace
passer par le correcteur  help
help
help  
help  
help  
help  
help  
help  
help  H
help  H
help  HE
help  HE
help  HEL
help  HEL
help  HELO
help  HELO
help  HELO
espace
passer par le correcteur  help
help
passer par le correcteur  HELO
HELO
espace
passer par le correcteur  help
help
passer par le correcteur  HELO
HELO
help  HELO  
help  HELO  
help  HELO  
help  HELO  
help  HELO  
help  HELO  B
help  HELO  B
help  HELO  B
help  HELO  B
help  HELO  B
help  HELO  B
help  HELO  B
help  HELO  B
help  HELO  BA
help  HELO  BA
help  HELO  BAB
help  HELO  BAB
help  HELO  BABL
help  HELO  BABL
espace
passer par le correcteur  help
help
passer par le correcteur  HELO
HELO
passer par le correcteur  BABL
baby
help  HELO  baby  
help  HELO  baby  
help  HELO  baby  
help  HELO  baby  A
help  HELO  baby  A
help  HELO  baby  A
help  HELO  baby  A
help  HELO  baby  AB
help  HELO  ba

## text 

In [56]:
text

'W  OC  cocolo  wole  '